In [1]:
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

Se cargan los datos

In [30]:
filename = "../../data/outputs/dataset_20210101_20211231_paragraph"
df_original = pd.read_csv(f"{filename}.csv", sep=";", parse_dates=["date"])

(10827, 48)

Se eliminan los párrafos duplicados (los que hayan sido asignados a varios tickers) para no etiquetar los mismos varias veces y se hace shuffle sobre ellos antes de elegir qué párrafos etiquetar

In [ ]:
df_original.drop_duplicates(subset=["body", "title"], inplace=True)
df = shuffle(df_original, random_state=42)

Generación del dataframe de noticias a etiquetar

In [5]:
#Robert
idx = df.title.unique()[:100]
#Hector
# idx = df.title.unique()[100:200]
titles = idx.tolist()
df = df[df.title.isin(idx)]
df = df.reset_index(drop=True)
df.sort_values(by=["date", "pk_paragraph"], inplace=True)

(458, 48)


# Annotator

In [7]:
def get_data(idx):
    noticia = titles[idx]
    noticia_parrafos = df[df.title == noticia]
    sentences = noticia_parrafos.index.astype(str) + ' - ' + noticia_parrafos["pk_paragraph"].astype(str) + ' - ' + noticia_parrafos['body']
    BATCH = len(sentences)
    return noticia, sentences, BATCH

In [34]:
contador = 0
contador_selector = 0
noticia, sentences, BATCH = get_data(contador)

title = widgets.HTML(value=f"Title: {noticia}")
display(title)

sentence_widget = widgets.Select(
    options=sentences,
    value=sentences.values[contador_selector],
    rows=BATCH,
    description='Párrafos',
    disabled=False,
    layout={'width': 'max-content'}
)
display(sentence_widget)

topic = widgets.Dropdown(
    options=[('Gobierno Corporativo +', 1),
    ('Gobierno Corporativo -', 2), 

    ('Reporte Beneficios +', 3),
    ('Reporte Beneficios -', 4),

    ('Financiero +', 5),
    ('Financiero -', 6),

    ('General +', 7),
    ('General -', 8),

    ('Legal +', 9),
    ('Legal -', 10),

    ('Operacional +', 11),
    ('Operacional -', 12),

    ('Restructuración +', 13),
    ('Restructuración -', 14),
    ('Duda', 15),],
    value=15,
    description='Topic noticia:',
)
display(topic)

finanzas_news = widgets.Dropdown(
    options=[('Sí', 0),
    ('No', 1), 
    ('Duda', 2)],
    value=2,
    description='Finanzas noticia:',
)
display(finanzas_news)

impacto_news = widgets.Dropdown(
    options=[('Sí', 0),
    ('No', 1), 
    ('Duda', 2),],
    value=2,
    description='Impacto noticia:',
)
display(impacto_news)

finanzas = widgets.Dropdown(
    options=[('Sí', 0),
    ('No', 1), 
    ('Duda', 2),],
    value=2,
    description='Finanzas:',
)
display(finanzas)
impacto = widgets.Dropdown(
    options=[('Sí', 0),
    ('No', 1), 
    ('Duda', 2),],
    value=2,
    description='Impacto:',
)
display(impacto)


progress = widgets.IntProgress(
    min=0,
    max=df.shape[0],
    step=1,
    description='Progreso:',
    orientation='horizontal',
    layout= widgets.Layout(width='20%')
)

display(progress)    

button = widgets.Button(description="Anotar", disabled=False, button_style="success", tooltip="Annotate")
output = widgets.Output()
button_next = widgets.Button(description="Siguiente", disabled=False)
display(button, button_next)
display(output)

def annotate(_):

    with output:
        finanzas_news_value = finanzas_news.value
        impacto_news_value = impacto_news.value
        topic_value = topic.value
        finanzas_value = finanzas.value
        impacto_value = impacto.value
        selected_sentence = sentence_widget.value
        
        if not len(selected_sentence):
            print("Error: No has seleccionado un párrafo")
            return                    
                 
        counter = int(selected_sentence.split(" - ")[0])
        df.loc[df.title == noticia, 'finanzas_tag_news_s_n'] = finanzas_news_value
        df.loc[df.title == noticia, 'impacto_tag_news_s_n'] = impacto_news_value
        df.loc[df.title == noticia,'topic_tag_news'] = topic_value
        df.loc[counter,'finanzas_tag_s_n'] = finanzas_value
        df.loc[counter,'impacto_tag_s_n'] = impacto_value

        print("counter", counter,
              "finanzas_tag_news_s_n", finanzas_news_value,
              "impacto_tag_news_s_n", impacto_news_value,
              "topic_tag_news", topic_value,
              "finanzas_tag_s_n", finanzas_value,
              "impacto_tag_s_n", impacto_value,
              )

        global contador_selector
        contador_selector += 1

        if contador_selector < BATCH:
            sentence_widget.value=sentences.values[contador_selector]

        if contador_selector <= BATCH:
            progress.value += 1
        

def on_button_clicked(_):
    global contador, contador_selector, BATCH, sentences, noticia
    contador_selector = 0
    contador += 1
    noticia, sentences, BATCH = get_data(contador)
    title.value=f"Title: {noticia}"
    sentence_widget.options = sentences
    sentence_widget.rows = BATCH
    topic.value=15
    finanzas_news.value=2
    impacto_news.value=2
    finanzas.value=2
    impacto.value=2
    with output:
        clear_output()

button.on_click(annotate)
button_next.on_click(on_button_clicked)

HTML(value='Title: Un grito de alerta en Casa Almirall')

Select(description='Párrafos', layout=Layout(width='max-content'), options=('49 - 1656201116281_6_ALM_00000000…

Dropdown(description='Topic noticia:', index=14, options=(('Gobierno Corporativo +', 1), ('Gobierno Corporativ…

Dropdown(description='Finanzas noticia:', index=2, options=(('Sí', 0), ('No', 1), ('Duda', 2)), value=2)

Dropdown(description='Impacto noticia:', index=2, options=(('Sí', 0), ('No', 1), ('Duda', 2)), value=2)

...

Dropdown(description='Impacto:', index=2, options=(('Sí', 0), ('No', 1), ('Duda', 2)), value=2)

IntProgress(value=0, description='Progreso:', layout=Layout(width='20%'), max=458)

Button(button_style='success', description='Anotar', style=ButtonStyle(), tooltip='Annotate')

Button(description='Siguiente', style=ButtonStyle())

Output()

Guardado del dataset etiquetado

In [37]:
notna_indexes = df.loc[:,["finanzas_tag_news_s_n", "impacto_tag_news_s_n", "topic_tag_news", "finanzas_tag_s_n", "impacto_tag_s_n"]].dropna().index
df_result = df.loc[notna_indexes,:]
df_result.to_csv(f"{filename}_annotated.csv", sep=";", index=False)